In [ ]:
import pandas as pd
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
import pickle

Preparing 2014 from excel

In [ ]:
df = pd.read_excel("Paragraphsyllabus2014.xlsx",sheet_name='Sheet1', usecols="E")
temp = df.values.tolist()
syllabus_2014 = []
for arr in temp:
  syllabus_2014.append(''.join(arr))
  
len(syllabus_2014)

60

Preparing 2018 from pickle

In [ ]:
subs = []
f = open("syllabus.pickle", "rb")
subs = pickle.load(f)
f.close()

syllabus_2018 = [] 
for des in subs:
    des = des.replace('.',',')
    sentences = des.split(".") 
    while '' in sentences:
        sentences.remove('')
    syllabus_2018.extend(sentences)
    
    
print("Total sentences: ",len(syllabus_2018)) 
    

Total sentences:  55


Utility functions

In [ ]:
def get_coherence(model, topics, docs):
  # Preprocess Documents
  documents = pd.DataFrame({"Document": docs,
                            "ID": range(len(docs)),
                            "Topic": topics})
  documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
  cleaned_docs = model._preprocess_text(documents_per_topic.Document.values)

  # Extract vectorizer and analyzer from BERTopic
  vectorizer = model.vectorizer_model
  analyzer = vectorizer.build_analyzer()

  # Extract features for Topic Coherence evaluation
  words = vectorizer.get_feature_names()
  tokens = [analyzer(doc) for doc in cleaned_docs]
  dictionary = corpora.Dictionary(tokens)
  corpus = [dictionary.doc2bow(token) for token in tokens]
  topic_words = [[words for words, _ in model.get_topic(topic)] 
                for topic in range(len(set(topics))-1)]

  # Evaluate
  coherence_model = CoherenceModel(topics=topic_words, 
                                  texts=tokens, 
                                  corpus=corpus,
                                  dictionary=dictionary, 
                                  coherence='c_v')
  coherence = coherence_model.get_coherence()
  return coherence


# Topic Modeling: 2014

In [ ]:
from bertopic import BERTopic

In [ ]:
 m_2014 = BERTopic(min_topic_size=3,top_n_words=20)
 t_2014, _ = m_2014.fit_transform(syllabus_2014)
 c_2014 = get_coherence(m_2014,t_2014,syllabus_2014)
 print(c_2014)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


0.43308802643722394


In [ ]:
m_2014.get_topic_info()

,Topic,Count,Name
0,0,27,0_memory_for_trees_circuits
1,1,13,1_software_testing_design_management
2,-1,7,-1_image_frequency_domain_authentication
3,2,7,2_data_distributed_sql_security
4,3,6,3_routing_wireless_layer_network


In [ ]:
m_2014.visualize_barchart(top_n_topics=10,n_words=10)

In [ ]:
m_2014.get_topics()

{-1: [('image', 0.07491461804291971),
  ('frequency', 0.050989991459343006),
  ('domain', 0.04812272827876048),
  ('authentication', 0.04400595819392057),
  ('models', 0.04386350073317427),
  ('filtering', 0.04249165954945251),
  ('data', 0.041570744261517294),
  ('algorithm', 0.03950149136028933),
  ('web', 0.037457309021459856),
  ('templates', 0.036687447959217585),
  ('shading', 0.036687447959217585),
  ('filters', 0.036687447959217585),
  ('segmentation', 0.035204766555136456),
  ('using', 0.034305840958628674),
  ('processing', 0.03129920731990643),
  ('ajax', 0.028949217559704456),
  ('vision', 0.028949217559704456),
  ('encryption', 0.028949217559704456),
  ('cipher', 0.028949217559704456),
  ('aspnet', 0.028949217559704456)],
 0: [('memory', 0.02897680596513289),
  ('for', 0.02653842276605827),
  ('trees', 0.02596894065306707),
  ('circuits', 0.02596894065306707),
  ('methods', 0.02436478617940983),
  ('search', 0.023971329833600372),
  ('retrieval', 0.023971329833600372),
  (

In [ ]:
m_2014.visualize_topics()

In [ ]:
m_2014.visualize_hierarchy()

# Topic Modeling: 2018

In [ ]:
 m_2018 = BERTopic(min_topic_size=3,top_n_words=20)
 t_2018, _ = m_2018.fit_transform(syllabus_2018)
 c_2018 = get_coherence(m_2018,t_2018,syllabus_2018)
 print(c_2018)

0.38728287844883375


In [ ]:
m_2018.get_topic_info()

,Topic,Count,Name
0,0,15,0_data_algorithm_algorithms_retrieval
1,-1,14,-1_for_parsing_languages_design
2,1,10,1_security_services_cloud_wireless
3,2,8,2_opencl_memory_system_organization
4,3,8,3_software_java_testing_test


In [ ]:
m_2018.visualize_barchart(top_n_topics=10,n_words=10)

In [ ]:
m_2018.get_topics()

{-1: [('for', 0.043928658841246845),
  ('parsing', 0.029316390236984218),
  ('languages', 0.02792500410252305),
  ('design', 0.027414595138176748),
  ('grammars', 0.026380383739802078),
  ('using', 0.02498802510491957),
  ('code', 0.023439638996911344),
  ('application', 0.023439638996911344),
  ('cipher', 0.02333264921297642),
  ('pointers', 0.02333264921297642),
  ('analysis', 0.023008668262371714),
  ('erlang', 0.023002928978427515),
  ('study', 0.021987292677738163),
  ('functions', 0.02187005637246671),
  ('applications', 0.02187005637246671),
  ('planning', 0.02015440262471582),
  ('regular', 0.019443874344147016),
  ('basic', 0.019047579539363847),
  ('tree', 0.018322743898115133),
  ('binary', 0.018322743898115133)],
 0: [('data', 0.04499191823327454),
  ('algorithm', 0.04486787656718304),
  ('algorithms', 0.038145274640760896),
  ('retrieval', 0.03689021855800769),
  ('image', 0.03610512646736251),
  ('learning', 0.03398228783970316),
  ('classification', 0.030982643059349876)

In [ ]:
m_2018.visualize_hierarchy()

In [ ]:
m_2018.visualize_topics()